# CRF Tutorial using python-crfsuite

In this tutorial, we will try to use CRF to work on part-of-speech (POS) tagging. There are 6 main parts in this tutorial
1. Setup and preprocessing
2. Designing feature funcions
3. Training
4. Making predictions
5. Evaluation
6. Try: Design a more complex model

# 1. Setup and preprocessing

In this demo we will use [python-crfsuite](https://github.com/scrapinghub/python-crfsuite)



In [6]:
import pycrfsuite
import numpy

We use POS data from [ORCHID corpus](https://www.nectec.or.th/corpus/index.php?league=pm), which is a POS corpus for Thai language.
A method used to read the corpus into a list of sentences with (word, POS) pairs have been implemented already. The example usage has shown below.

In [2]:
from data.orchid_corpus import get_sentences
train_data = get_sentences('train')
test_data = get_sentences('test')
train_data[0]

[('การ', 'FIXN'),
 ('ประชุม', 'VACT'),
 ('ทาง', 'NCMN'),
 ('วิชาการ', 'NCMN'),
 ('<space>', 'PUNC'),
 ('ครั้ง', 'CFQC'),
 ('ที่ 1', 'DONM')]

## 2. Designing features functions

- __word2features()__: This method returns all feature functions for time step _i_ of an input sequence. So, this method is where all feature functions are defined. From the code, we can define just features from input sequence (word for this example), the library will manage the transition functions ($y_{t-1}$ -> $y_t$) and state functions ($y_t$ -> $X$, with all $X$ features you defined in this method) for you.
- __sent2features()__: Loop and call word2features() over the input sequence.
- __sent2labels()__: Get the output labels from train/test sequence
- __sent2tokens()__: Get words from train/test sequence (used in prediction part just to show the full result)

In [3]:
def word2features(sent, i):
    word = sent[i][0]
    
    features = {
        'word': word,
        'word.isdigit': word.isdigit(),
        'word.length': len(word),
    }
    
    features['BOS'] = (i == 0)  # beginning of sentence
    features['EOS'] = (i == len(sent)-1)  # end of sentence
    
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for (word, label) in sent]

def sent2tokens(sent):
    return [word for (word, label) in sent]

In [4]:
sent2features(train_data[0])[0]

{"  'word.firstLetter': word[0],\n        'word.lastLetter': word[-1], word.length": 3,
 'BOS': True,
 'EOS': False,
 'word': 'การ',
 'word.isdigit': False}

In [5]:
%%time
x_train = [sent2features(sent) for sent in train_data]
y_train = [sent2labels(sent) for sent in train_data]
x_test = [sent2features(sent) for sent in test_data]
y_test = [sent2labels(sent) for sent in test_data]

CPU times: user 343 ms, sys: 84.4 ms, total: 427 ms
Wall time: 427 ms


## 3. Training

To train a CRF model in python-crfsuite, we have to create a trainer and load training data (pairs of __generated features__ and __labels__) to the trainer first.

In [6]:
trainer = pycrfsuite.Trainer(verbose=False)
for xseq, yseq in zip(x_train, y_train):
    trainer.append(xseq, yseq)

There are several parameters you can set for the training process. You can list all parameter using this method.

In [7]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In this tutorial, we will use 3 parameters:

- __max_iterations__: Define how many times we will let the model learn through training data
- __feature.possible_transitions__: Enable the library to create transition feature functions (as we discussed in section 2)
- __feature.possible_states__: Enable state feature functions

In [8]:
trainer.set_params({
    'max_iterations': 100,
    'feature.possible_transitions': True,
    'feature.possible_states': True,
})

Finally, call the trainer to train with the specified model path.

In [9]:
%%time
model_path = 'model/crf_basic.model'
trainer.train(model_path)

CPU times: user 5min 44s, sys: 265 ms, total: 5min 45s
Wall time: 5min 44s


## 4. Making predictions

When we finished training a model. We can use that model to predict any sequence of words.
To do this, create a tagger with path to the saved model. Then, generate features with a sequence we want to predict and send them to _tag_ method.

In [10]:
tagger = pycrfsuite.Tagger()
tagger.open(model_path)

In [11]:
example_sent = test_data[20]
print(' '.join(sent2tokens(example_sent)))

print('Predicted: ', ' '.join(tagger.tag(sent2features(example_sent))))
print('Correct: ', ' '.join(sent2labels(example_sent)))

<minus> <space> ระบบ การ บันทึก รหัส ไว้ ใน แฟ้มข้อมูล
Predicted:  PUNC PUNC NCMN FIXN VACT NCMN XVAE RPRE NCMN
Correct:  PUNC PUNC NCMN FIXN VACT NCMN XVAE RPRE NCMN


## 5. Evaluation

To measure how good the model can perform, we have to evaluate the model on _test data_. For sequence labeling tasks, we often use __accuracy__ to measure a model's goodness. However, we can analyze further by considering each tag with
- __prediction__: How many times the predicted tag _x_ is correctly tagged (it is a tag _x_ in the test data)
- __recall__: How many times the real tag _x_ is correctly tagged (the model can answer that it is a tag _x_)

The method below, evaluation_report(), is implemented to measure all metrics described and display it in DataFrame. It is ok to just use this method and not going through this.

In [15]:
import pandas as pd
from IPython.display import display

def evaluation_report(y_true, y_pred):
    # retrieve all tags in y_true
    tag_set = set()
    for sent in y_true:
        for tag in sent:
            tag_set.add(tag)
    tag_list = sorted(list(tag_set))
    
    # count correct points
    tag_info = dict()
    for tag in tag_list:
        tag_info[tag] = {'correct_tagged': 0, 'y_true': 0, 'y_pred': 0}

    all_correct = 0
    all_count = sum([len(sent) for sent in y_true])
    for sent_true, sent_pred in zip(y_true, y_pred):
        for tag_true, tag_pred in zip(sent_true, sent_pred):
            if tag_true == tag_pred:
                tag_info[tag_true]['correct_tagged'] += 1
                all_correct += 1
            tag_info[tag_true]['y_true'] += 1
            tag_info[tag_pred]['y_pred'] += 1
    accuracy = (all_correct / all_count) * 100
            
    # summarize and make evaluation result
    eval_list = list()
    for tag in tag_list:
        eval_result = dict()
        eval_result['tag'] = tag
        eval_result['correct_count'] = tag_info[tag]['correct_tagged']
        precision = (tag_info[tag]['correct_tagged']/tag_info[tag]['y_pred'])*100 if tag_info[tag]['y_pred'] else '-'
        recall = (tag_info[tag]['correct_tagged']/tag_info[tag]['y_true'])*100
        eval_result['precision'] = precision
        eval_result['recall'] = recall
        eval_result['f_score'] = (2*precision*recall)/(precision+recall) if (type(precision) is float and recall > 0) else '-'
        
        eval_list.append(eval_result)

    eval_list.append({'tag': 'accuracy=%.2f' % accuracy, 'correct_count': '', 'precision': '', 'recall': '', 'f_score': ''})
    
    df = pd.DataFrame.from_dict(eval_list)
    df = df[['tag', 'precision', 'recall', 'f_score', 'correct_count']]
    display(df)

Make predictions on test set (y_pred) and evaluate against the real label (y_test)

In [13]:
y_pred = [tagger.tag(x_sent) for x_sent in x_test]

In [14]:
evaluation_report(y_test, y_pred)

,tag,precision,recall,f_score,correct_count
0,ADVI,-,0,-,0
1,ADVN,69.4611,20.6774,31.8681,232
2,ADVP,-,0,-,0
3,ADVS,-,0,-,0
4,CFQC,-,0,-,0
5,CLTV,-,0,-,0
6,CMTR,13.0435,1.45278,2.61438,6
7,CMTR@PUNC,-,0,-,0
8,CNIT,100,3.53261,6.82415,13
9,DCNM,69.9789,72.3497,71.1445,662


## 6. Use pretrained word embedding

In this exercise, we will use pretrained word embedding from previous homework as word feature in pycrfsuite. We load pretrained word embedding using pickle. The pretrained weight is a dictionary which map a word to its embedding.

In [3]:
import pickle
fp = open('basic_ff_embedding.pt', 'rb')
embeddings = pickle.load(fp)
fp.close()

In [20]:
def word2features(sent, i, emb):
    def add_embedding_features(feat, prefix, query_word):
        if query_word in emb:
            vec = emb[query_word]
        else:
            vec = numpy.zeros(32)
        
        for i in range(vec.shape[0]):
            feat[prefix + str(i)] = vec[i]
    
    features = dict()
    word = sent[i][0]
    add_embedding_features(features, 'word.embd', word)
    features.update({
        'word.word' : word,
        'word.isdigit': word.isdigit(),
        'word.length': len(word),
    })
    
    features['BOS'] = (i == 0)  # beginning of sentence
    features['EOS'] = (i == len(sent)-1)  # end of sentence
    
    return features

def sent2features(sent, emb_dict):
    return [word2features(sent, i, emb_dict) for i in range(len(sent))]

def sent2labels(sent):
    return [label for (word, label) in sent]

def sent2tokens(sent):
    return [word for (word, label) in sent]

In [21]:
%%time
x_train = [sent2features(sent, embeddings) for sent in train_data]
y_train = [sent2labels(sent) for sent in train_data]
x_test = [sent2features(sent, embeddings) for sent in test_data]
y_test = [sent2labels(sent) for sent in test_data]

CPU times: user 6.36 s, sys: 452 ms, total: 6.82 s
Wall time: 6.82 s


In [22]:
sent2features(train_data[0], embeddings)[0]

{'BOS': True,
 'EOS': False,
 'word.embd0': 0.63079655,
 'word.embd1': 0.55423963,
 'word.embd10': -0.52982348,
 'word.embd11': 0.74475533,
 'word.embd12': 0.68278229,
 'word.embd13': -0.5775221,
 'word.embd14': -0.66996753,
 'word.embd15': 0.66535348,
 'word.embd16': -0.64394021,
 'word.embd17': 0.62942129,
 'word.embd18': -0.68831235,
 'word.embd19': -0.66224283,
 'word.embd2': -0.69944656,
 'word.embd20': -0.82274407,
 'word.embd21': -0.59909046,
 'word.embd22': 0.66668463,
 'word.embd23': 0.65602303,
 'word.embd24': 0.68236977,
 'word.embd25': 0.57738519,
 'word.embd26': -0.54963934,
 'word.embd27': 0.60753429,
 'word.embd28': -0.67751026,
 'word.embd29': -0.60196757,
 'word.embd3': 0.66754633,
 'word.embd30': 0.55823398,
 'word.embd31': 0.56031388,
 'word.embd4': 0.71997637,
 'word.embd5': 0.56522852,
 'word.embd6': -0.59826338,
 'word.embd7': 0.58731371,
 'word.embd8': 0.64380872,
 'word.embd9': 0.52099121,
 'word.isdigit': False,
 'word.length': 3,
 'word.word': 'การ'}

In [23]:
%%time
trainer = pycrfsuite.Trainer(verbose=True)
trainer.set_params({
    'max_iterations': 100,
    'feature.possible_transitions': True,
    'feature.possible_states': True,
})

for xseq, yseq in zip(x_train, y_train):
    trainer.append(xseq, yseq)

CPU times: user 6.35 s, sys: 0 ns, total: 6.35 s
Wall time: 6.35 s


In [24]:
%%time
model_path = 'model/crf_neural.model'
trainer.train(model_path)

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 1
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 709136
Seconds required: 55.079

L-BFGS optimization
c1: 0.000000
c2: 1.000000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 776867.117349
Feature norm: 1.000000
Error norm: 282725.103522
Active features: 709136
Line search trials: 1
Line search step: 0.000001
Seconds required for this iteration: 5.668

***** Iteration #2 *****
Loss: 741788.669404
Feature norm: 0.979262
Error norm: 413664.190556
Active features: 709136
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 2.896

***** Iteration #3 *****
Loss: 709224.554318
Feature norm: 0.996845
Error norm: 263743.801395
Active features: 709136
Line search trials: 1
Line search step: 1.000000
Seconds r

In [25]:
%%time
model_path = 'model/crf_neural.model'
tagger = pycrfsuite.Tagger()
tagger.open(model_path)
y_pred = [tagger.tag(x_sent) for x_sent in x_test]

CPU times: user 2.63 s, sys: 4.01 ms, total: 2.64 s
Wall time: 2.64 s


In [26]:
evaluation_report(y_test, y_pred)

,tag,precision,recall,f_score,correct_count
0,ADVI,-,0,-,0
1,ADVN,53.5484,36.9875,43.7533,415
2,ADVP,-,0,-,0
3,ADVS,-,0,-,0
4,CFQC,-,0,-,0
5,CLTV,-,0,-,0
6,CMTR,48.6239,12.8329,20.3065,53
7,CMTR@PUNC,-,0,-,0
8,CNIT,39.3617,20.1087,26.6187,74
9,DCNM,69.715,64.153,66.8184,587
